In [33]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/Splycer")
from record_set import RecordDataFrame
from compare_set import CompareCSR
from feature_engineer import FeatureEngineer
from xgboost_match import XGBoostMatch
os.chdir("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/example")

In [20]:
#Load in the census data.
delaware_1910 = RecordDataFrame(2, pd.read_csv("delaware_1910.csv", index_col="index"))
delaware_1920 = RecordDataFrame(3, pd.read_csv("delaware_1920.csv", index_col="index"))

In [25]:
#Load in the compares set
compares = pd.read_csv("delaware_compares.csv")
compares = CompareCSR([1,2], compares.index_1910.values, compares.index_1920.values, np.ones(compares.shape[0]))

In [28]:
#Construct the feature engineer
years = [1,2]
feature_engineer = FeatureEngineer()
features = ["imputer", "bool match", "euclidean distance", "jaro-winkler", 
            "ngram", "drop", "commonality weight"] #FIXME add immigration imputer to FeatureEngineer
first_vec = [f"first_vec{i}" for i in range(2, 202)]
last_vec = [f"last_vec{i}" for i in range(2,202)]
cols_to_drop = ["marstat", "birth_year", "household", "immigration", "race", "rel", "female",
                "bp", "mbp", "fbp", "state", "county", "cohort1", 
                "cohort2", "last_sdxn", "first_sdxn", "last_init",
                "first_init", "bp_comm", "first_comm", "last_comm", "bp_lat", "bp_lon",
                "res_lat", "res_lon", "full_name"]
cols_to_drop.extend(first_vec)
cols_to_drop.extend(last_vec)
feature_params = [{"cols": ["first_comm", "last_comm"], "years": years}, 
                  {"vars_to_match": ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county"], "years": years},
                  {"variables": [["immigration"], ["birth_year"], ["res_lat", "res_lon"], ["bp_lat", "bp_lon"], [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)]], "new_cols": ["immigration_dist", "birth_year_dist", "geodist", "bp_geodist", "first_vec_dist", "last_vec_dist"], "years": years},
                  {"string_col": ["first", "last"], "dist_metric": "jw", "years": years},
                  {"string_col": ["first", "last"], "n": 2, "years": years},
                  {"cols_to_drop": ["first", "last"], "both_years": True, "years": years},
                  {"cols": ["first_jw", "last_jw", "first_sdxn_match", "last_sdxn_match", "bp_match"], "comm_cols": ["first", "last", "first", "last", "bp"], "years": years}]

for i, j in zip(features, feature_params):
    feature_engineer.add_feature(i, j) #this adds features to your feature object. when you run feature_engineer.create_features(), it computes all of the features above.
feature_engineer.build()

In [37]:
#Construct XGBoostMatch
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/model.xgboost", "r") as file:
    model = pkl.load(file)
xbg = XGBoostMatch(delaware_1910, delaware_1920, compares, feature_engineer, model)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 36: character maps to <undefined>